__собираем предложения о земельных участках__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# каталог с файлами объявлений
avito_raw_data_path = 'data/raw/land/avito' 
# cian_raw_data_path = 'data/raw/cian'
result_file_path = 'data/data_land.pkl'

In [5]:
# ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
# ts

## собираем данные авито.ру

In [6]:
import re
from os import listdir

raw_data_files = sorted([
        avito_raw_data_path+'/'+f 
        for f in listdir(avito_raw_data_path) 
        if re.match(r'.+\.xlsx$',f)
    ])
raw_data_files

['data/raw/land/avito/avito_2022-09-08_16-05_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-12_13-51_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-13_13-56_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-15_10-31_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-16_15-10_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-19_12-39_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-22_09-57_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-26_11-22_raw.xlsx',
 'data/raw/land/avito/avito_2022-10-08_20-38_raw.xlsx',
 'data/raw/land/avito/avito_2022-10-14_15-54_raw.xlsx',
 'data/raw/land/avito/avito_2022-11-01_13-58_raw.xlsx',
 'data/raw/land/avito/avito_2022-11-04_17-12_raw.xlsx',
 'data/raw/land/avito/avito_2022-11-07_10-46_raw.xlsx',
 'data/raw/land/avito/avito_2022-11-15_13-16_raw.xlsx',
 'data/raw/land/avito/avito_2022-11-28_14-58_raw.xlsx',
 'data/raw/land/avito/avito_2023-01-10_11-27_raw.xlsx',
 'data/raw/land/avito/avito_2023-01-27_16-42_raw.xlsx']

In [7]:
# for f in raw_data_files:
#     df = pd.read_excel(f)
#     df['place']='sevastopol'
#     df.to_excel(f,index=False)
#     print(f)

In [8]:
data_avito = pd.concat([
        pd
        .read_excel(f)
        #.dropna()
        .drop_duplicates(['avito_id','description'])
        .rename(columns={'page':'avito_page'})
        .reset_index(drop=True)  
        for f in raw_data_files
    ]).reset_index(drop=True)

print(len(data_avito))

17940


In [9]:
# data_avito

In [10]:
from lib.avito import AvitoDataCleanerRealtyLand

data_avito = AvitoDataCleanerRealtyLand().transform( data_avito )

print(len(data_avito))

17940


In [11]:
data_avito.sample(3)

,avito_id,title,price,obj_name,adr,description,avito_page,ts,place,area,is_IJS,priceM,priceMU,area_size_category
2088,2490769490,Участок 8 сот. (ИЖС),8000000,,садовое товарищество Поиск,Электричество есть. За водопровод проплачено. На участке есть металлический гараж.,21,2022-09-12 13:51:16.745,sevastopol,8.00,True,8.00,1.00,4-8
8156,2410302762,Участок 24 сот. (ИЖС),13999999,,Фёдоровская ул.,"Продажа От Собственника.\n\nСевастополь, а в особенности его Северную сторону, абсолютно не затапливает ни при каких ливнях! Не зря расположение аэродрома «Бельбек» выбрано именно на Северной стороне. \n\nУчасток, 600 м от моря. Пляж Любимовка (песчано-галечны",9,2022-09-26 11:22:23.724,sevastopol,24.00,True,14.00,0.58,20+
9071,2443820760,Участок 10 сот. (ИЖС),5000000,,Гагаринский муниципальный округ,"Срочно! Продам участок √8, в хорошем месте. Не далеко от дороги, до остановки 500 метров. Участок расположен в Снт «Рябинушка», перед участком красивый сосновый лес. Участок 10 соток, форма буква Г. \nУчасток подходит для строительства дома или же элитного",6,2022-10-08 20:38:42.759,sevastopol,10.00,True,5.00,0.50,8-20


In [12]:
assert len(data_avito)>0

In [13]:
data_avito.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17940 entries, 0 to 17939
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   avito_id            17940 non-null  int64         
 1   title               17940 non-null  object        
 2   price               17940 non-null  int64         
 3   obj_name            17940 non-null  object        
 4   adr                 17026 non-null  object        
 5   description         17940 non-null  object        
 6   avito_page          17940 non-null  int64         
 7   ts                  17940 non-null  datetime64[ns]
 8   place               17940 non-null  object        
 9   area                17940 non-null  float64       
 10  is_IJS              17940 non-null  bool          
 11  priceM              17940 non-null  float64       
 12  priceMU             17940 non-null  float64       
 13  area_size_category  17930 non-null  category  

---

In [14]:
data_avito['url'] = 'https://avito.ru/' + data_avito['avito_id'].astype(int).astype(str)
data_avito = data_avito.drop(columns=['avito_id'])
# data_cian = data_cian.rename(columns={'cian_url':'url'})

In [15]:
data = data_avito.reset_index(drop=True) 
print(len(data))
assert len(data)>0

17940


## обновляем таблицу адресов

In [16]:
# data_ = data.sample(15).reset_index(drop=True)
# data_[['adr']]

In [17]:
places = { 
# поле 'place' и название города
'bahchisaray': 'Бахчисарай,',
      'sudak': 'Судак,',
  'sevastopol': 'Севастополь,',
}

def norm_place(d,places=places):
    if not d['place'] in places: return d['adr_']
    p = places[ d['place'] ]
    return d['adr_'] if d['adr_'].find(p)>-1 else p+d['adr_']

In [18]:
# data[['adr']].drop_duplicates().sort_values('adr')
# .info()

In [19]:
from lib.locator import AddressTransformer

atr = AddressTransformer()
data['adr_'] = data['adr'].fillna('').progress_apply(atr.transform)
data['adr_'] = data[['adr_','place']].progress_apply(norm_place,axis=1)
data = data.rename(columns={'adr':'adr_orig','adr_':'adr'})

  0%|          | 0/17940 [00:00<?, ?it/s]

  0%|          | 0/17940 [00:00<?, ?it/s]

In [20]:
data[['adr_orig','adr']].sample(10)

,adr_orig,adr
14885,"пр. Артиллеристов, 60А","Севастополь,пр. Артиллеристов, 60А"
9457,с. Гончарное,"Севастополь,с. Гончарное"
3462,"Крепостное ш., 31/1","Севастополь,Крепостное ш., 31/1"
14568,Камышовое ш.,"Севастополь,Камышовое ш."
17775,NaN,"Севастополь,"
1998,"садоводческое товарищество Кальфа-1, 24","Севастополь,садоводческое товарищество Кальфа-1, 24"
14496,с. Осипенко,"Севастополь,с. Осипенко"
13606,"1-я Диагональная ул., 16","Севастополь,1-я Диагональная ул., 16"
1991,"с. Вишнёвое, ул. Мира, 32","Севастополь,с. Вишнёвое, ул. Мира, 32"
2583,"Румянцевская ул., 5","Севастополь,Румянцевская ул., 5"


In [21]:
# from lib.locator import GeocoderSimpleOSM
from lib.locator import GeocoderYandex
from yandex_keys import keys

atr = AddressTransformer()
locator = GeocoderYandex(keys[0])
# locator = GeocoderSimpleOSM()

loc_file_path = 'data/location.pkl' # таблица адресов
loc = pd.read_pickle(loc_file_path)
# loc = pd.DataFrame([],columns=['adr','latitude','longitude',])
logging.info(f'LocationUpdater: {len(loc)} addresses in index')

# собираем все адреса в один список
loc = loc.merge(
        data[['adr']].drop_duplicates(),
        how='outer',
        on='adr',
    ).drop_duplicates().reset_index(drop=True)

logging.info(f'LocationUpdater: {len(loc)} addresses total')

# собираем все адреса с геопозицией
loc_def = loc[~loc['latitude'].isna()].reset_index(drop=True)
logging.info(f'LocationUpdater: {len(loc_def)} addresses defined')

# собираем все адреса без геопозиции
loc_undef = loc[loc['latitude'].isna()][['adr']].reset_index(drop=True)
logging.info(f'LocationUpdater: {len(loc_undef)} addresses undefined')

if len(loc_undef)>0:
    # определяем координаты по строке адреса
    loc_undef = loc_undef.join( loc_undef['adr'].progress_apply(locator.transform).apply(pd.Series) ).dropna()     
    logging.info(f'LocationUpdater: {len(loc_undef)} new addresses found')
    loc = pd.concat([loc_def,loc_undef]).reset_index(drop=True) 
    assert len(loc)>0
    loc.to_pickle(loc_file_path)

[INFO    ] 2023-01-27 16:57:14 | LocationUpdater: 8676 addresses in index
[INFO    ] 2023-01-27 16:57:14 | LocationUpdater: 8676 addresses total
[INFO    ] 2023-01-27 16:57:14 | LocationUpdater: 8676 addresses defined
[INFO    ] 2023-01-27 16:57:14 | LocationUpdater: 0 addresses undefined


In [22]:
print( len(loc) )
loc.sample(3)

8676


,latitude,longitude,adr
3944,44.56,33.46,"Севастополь,Византийская ул., 12"
500,44.61,33.55,"Севастополь, улица Хрулёва, 28"
2399,44.579403,33.507223,"Севастополь, район Ленинский, улица Токарева, 18Б"


## дополняем данные геометкой

In [23]:
# data = data.rename(columns={'adr':'adr_orig','adr_':'adr'})

data = data.merge(loc,on=['adr'],how='left')

print('всего записей:', len( data) )
print('записей без геометки:', len( data[ data['latitude'].isna() ] ) )

assert len(data)>0
data.to_pickle(result_file_path)

data.info()

всего записей: 17940
записей без геометки: 0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 17940 entries, 0 to 17939
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   title               17940 non-null  object        
 1   price               17940 non-null  int64         
 2   obj_name            17940 non-null  object        
 3   adr_orig            17026 non-null  object        
 4   description         17940 non-null  object        
 5   avito_page          17940 non-null  int64         
 6   ts                  17940 non-null  datetime64[ns]
 7   place               17940 non-null  object        
 8   area                17940 non-null  float64       
 9   is_IJS              17940 non-null  bool          
 10  priceM              17940 non-null  float64       
 11  priceMU             17940 non-null  float64       
 12  area_size_category  17930 non-null  category      
 13  u

In [25]:
data[['ts']].groupby('ts').size()

ts
2022-09-08 16:04:40.227    1077
2022-09-12 13:51:16.745    1106
2022-09-13 13:56:54.600    1111
2022-09-15 10:31:26.039    1121
2022-09-16 15:10:34.059    1100
2022-09-19 12:39:53.571    1104
2022-09-22 09:57:15.623    1109
2022-09-26 11:22:23.724    1087
2022-10-08 20:38:42.759    1109
2022-10-14 15:54:34.254    1101
2022-11-01 13:58:37.789    1062
2022-11-04 17:12:55.140     975
2022-11-07 10:46:32.363    1028
2022-11-15 13:16:46.237    1079
2022-11-28 14:58:16.679    1030
2023-01-10 11:27:43.921     827
2023-01-27 16:42:18.173     914
dtype: int64